In [ ]:
!pip install neattext

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import neattext as nt

In [ ]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [ ]:
print('Train data shape:', train_df.shape)
print('Train columns:', train_df.columns)
print('Test data shape:', test_df.shape)
print('Test columns:', test_df.columns)

In [ ]:
train_df = train_df.drop(['id'], axis = 1)
test_df = test_df.drop(['id'], axis = 1)
df = df.drop(['id'], axis = 1)

In [ ]:
print('Number of duplicated text on train data: ',train_df.duplicated(['text']).sum())
print('Number of duplicated text on test data: ',test_df.duplicated(['text']).sum())

In [ ]:
train_df[train_df['text'].duplicated() == True].sort_values(by=['text'])

In [ ]:
s = train_df.groupby('text').target.agg(lambda x : x.mode()[0])
drop_dup = pd.DataFrame({'text':s.index, 'target':s.values})
train_df = train_df.drop_duplicates(subset=['text'])
train_df.loc[train_df.text.isin(drop_dup.text), ['target']] = drop_dup[['target']]
train_df

In [ ]:
test_df = test_df.drop_duplicates(subset=['text'])
test_df

In [ ]:
def missing_ratio(_df):
    missing_cols = _df.columns[_df.isnull().any()]
    plt.subplots(figsize=(6, 6))
    missing_ratio = _df[missing_cols].isnull().sum() * 100 / len(_df)
    missing_ratio.sort_values(inplace=True)
    missing_ratio.plot.bar(missing_ratio)
    return missing_ratio

In [ ]:
train_missing = missing_ratio(train_df)
train_missing

In [ ]:
test_missing = missing_ratio(test_df)
test_missing

In [ ]:
def clean_text(text):
    docx = nt.TextFrame(text=text)
    docx.normalize(level='deep')
    docx.remove_puncts()
    docx.remove_urls()
    docx.remove_special_characters()
    docx.remove_emojis()
    docx.fix_contractions()
    return docx.text
    

In [ ]:
train_df['text']=train_df['text'].apply(lambda x: clean_text(x))
test_df['text']=test_df['text'].apply(lambda x: clean_text(x))

In [ ]:
train_df['text']

In [ ]:
test_df['text']